<a href="https://colab.research.google.com/github/ParitKansal/GenAI/blob/main/LangChain_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**LangChain**

LangChain is a framework for developing applications powered by language models.

- GitHub: https://github.com/hwchase17/langchain
- Docs: https://python.langchain.com/v0.2/docs/introduction/

### Overview:
- Installation
- LLMs
- Prompt Templates
- Chains
- Agents and Tools
- Memory
- Document Loaders
- Indexes

#**01: Installation**

In [1]:
!pip install langchain langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 34.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 34.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.7 MB/s eta 0:00:00
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.36
    Uninstalling SQLAlchemy-2.0.36:
      Successfully uninstalled SQLAlchemy-2.0.36


#**02: Setup the Environment**

In [1]:
import os
from google.colab import userdata

In [2]:
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

os.environ["HUGGINGFACEHUB_API_TOKEN"] = userdata.get('HUGGINGFACEHUB_API_TOKEN')

# **03: Large Language Models**

The basic building block of LangChain is a Large Language Model which takes text as input and generates more text

Suppose we want to generate a company name based on the company description, so we will first initialize an OpenAI wrapper. In this case, since we want the output to be more random, we will intialize our model with high temprature.

The temperature parameter adjusts the randomness of the output. Higher values like 0.7 will make the output more random, while lower values like 0.2 will make it more focused and deterministic.

temperature value--> how creative we want our model to be

0 ---> temperature it means model is  very safe it is not taking any bets.

1 --> it will take risk it might generate wrong output but it is very creative

## **Open AI**

In [6]:
!pip install openai

In [9]:
%pip install -qU langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 40.5 MB/s eta 0:00:00


In [10]:
from langchain_openai import OpenAI
llm = OpenAI(temperature=0.9)

And now we will pass in text and get  predictions

In [16]:
text="I want to open a restaurant for Chinese food. Suggest a fency name for this."

In [17]:
print(llm.invoke(text))



"Imperial Dragon Palace" 


## **Hugging Face**

In [20]:
pip install langchain-huggingface

In [22]:
from langchain_huggingface import HuggingFaceEndpoint

In [42]:
llm = HuggingFaceEndpoint(
    repo_id="mistralai/Mistral-7B-Instruct-v0.2",
    max_length=5,
    temperature=0.5,
)
question = "Who won the FIFA World Cup in the year 1994? "
print(llm.invoke(question))

                    max_length was transferred to model_kwargs.
                    Please make sure that max_length is what you intended.


1994 FIFA World Cup was won by Brazil. Brazil won the tournament by defeating Italy in the final match, which took place on July 17, 1994. This was Brazil's fourth World Cup title, and they won it under the leadership of their legendary coach, Carlos Alberto Parreira. The final match was held at the Rose Bowl in Pasadena, California, United States. Brazil won the match 0-0 in regular time, and won the title in the penalty shootout. The team's star players included Romário, Ronaldo (Ronaldinho's elder brother), Dunga, and Cafu, among others.


# **04: Prompt Templates**

Currently in the above applications we are writing an entire prompt, if you are creating a user directed application then this is not an ideal case

LangChain faciliates prompt management and optimization.

Normally when you use an LLM in an application, you are not sending user input directly to the LLM. Instead, you need to take the user input and construct a prompt, and only then send that to the LLM.

In many Large Language Model applications we do not pass the user input directly to the Large Language Model, we add the user input to a large piece of text called prompt template

## **Example 1**

In [51]:
from langchain.prompts import PromptTemplate

prompt_template_name = PromptTemplate(
    input_variables=['cuisine', 'count'],  # List of input variable names
    input_types={'cuisine': str, 'count': int},  # Types of the input variables
    template="I want to open a restaurant for {cuisine} food. Suggest {count} fancy names for this."
)

p = prompt_template_name.format(cuisine="indian", count=2)
print(p)

I want to open a restaurant for indian food. Suggest 2 fancy names for this.


## **Example 2**

In [56]:
from langchain.prompts import PromptTemplate

prompt_template_name = PromptTemplate(
    input_variables=['cuisine', 'count'],  # List of input variable names
    input_types={'cuisine': str, 'count': int},  # Types of the input variables
    template="I want to open a restaurant for {cuisine} food. Suggest {count} fancy names for this."
)

def format_with_defaults(cuisine="indian", count=2):
    return prompt_template_name.format(cuisine=cuisine, count=count)

p = format_with_defaults("Chinese", "5")
print(p)

I want to open a restaurant for Chinese food. Suggest 5 fancy names for this.


# **05: Chains**

Combine LLMs and Prompts in multi-step workflows

Now as we have the  **model**:

```
llm = OpenAI(temperature=0.9)
```


and the **Prompt Template**:
```
prompt =  PromptTemplate(
    input_variables=['cuisine', 'count'],  # List of input variable names
    input_types={'cuisine': str, 'count': int},  # Types of the input variables
    template="I want to open a restaurant for {cuisine} food. Suggest {count} fancy names for this."
)
prompt.format(cuisine="indian", count=2)
```

Now using Chains we will link together model and the PromptTemplate and other Chains

The simplest and most common type of Chain is LLMChain, which passes the input first to Prompt Template and then to Large Language Model

LLMChain is responsible to execute the PromptTemplate, For every PromptTemplate we will specifically have an LLMChain

### **Example 1**

In [57]:
from langchain_openai import OpenAI
llm = OpenAI(temperature=0.9)

In [58]:
from langchain.prompts import PromptTemplate
prompt =  PromptTemplate(
    input_variables=['cuisine', 'count'],  # List of input variable names
    input_types={'cuisine': str, 'count': int},  # Types of the input variables
    template="I want to open a restaurant for {cuisine} food. Suggest {count} fancy names for this."
)
prompt.format(cuisine="indian", count=2)

'I want to open a restaurant for indian food. Suggest 2 fancy names for this.'

Whatever input text i am giving that will get assigned to this particular variable that is **product**

In [68]:
from langchain.chains import LLMChain

chain = LLMChain(llm=llm, prompt=prompt)
response= chain.run(cuisine = "indian", count = 2)
response

'\n\n1. "TajMahal Bites" \n2. "Spice Palace Dining"'

### **Example 2**

In [73]:
chain = LLMChain(llm=llm, prompt=prompt, verbose=True)
response=chain.run(cuisine = "Mexican", count = 2)
print(response)



> Entering new LLMChain chain...
Prompt after formatting:
I want to open a restaurant for Mexican food. Suggest 2 fancy names for this.

> Finished chain.


1. "El Sabor Exótico" (The Exotic Flavor)
2. "La Fiesta de Sabores" (The Flavor Party)


## **To combine the Chain and  to set a sequence for that we use SimpleSequentialChain**

**Can we combine Multiple PromptTemplates, We will try to combine Multiple PromptTemplates**

**The output from the first PromptTemplate is passed to the next PromptTemplate as input**

###**Simple Sequential Chain**

In [78]:
from langchain_openai import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

llm = OpenAI(temperature=0.6)

prompt_template_name = PromptTemplate(
    input_variables =['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest a fency name for this."
)
name_chain =LLMChain(llm=llm, prompt=prompt_template_name, verbose = True)

prompt_template_items = PromptTemplate(
    input_variables = ['restaurant_name'],
    template="""Suggest some menu items for {restaurant_name}"""
)
food_items_chain = LLMChain(llm=llm, prompt=prompt_template_items,verbose = True)

In [80]:
from langchain.chains import SimpleSequentialChain
chain = SimpleSequentialChain(chains = [name_chain, food_items_chain], verbose = True)

content = chain.run("indian")



> Entering new SimpleSequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
I want to open a restaurant for indian food. Suggest a fency name for this.

> Finished chain.


"Spice Symphony"


> Entering new LLMChain chain...
Prompt after formatting:
Suggest some menu items for 

"Spice Symphony"

> Finished chain.


1. Tandoori Chicken Tikka
2. Lamb Vindaloo
3. Vegetable Samosas
4. Butter Chicken
5. Palak Paneer
6. Masala Dosa
7. Garlic Naan
8. Chana Masala
9. Mango Lassi
10. Gulab Jamun (dessert)

> Finished chain.


**There is a issue with SimpleSequentialChain it only shows last input information**

## **To show the entire information i will use SequentialChain(Good)**

### **Sequential Chain**

In [124]:
llm = OpenAI(temperature=0.7)

prompt_template_name = PromptTemplate(
    input_variables =['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest a fency name for this."
)
name_chain =LLMChain(llm=llm, prompt=prompt_template_name, output_key="restaurant_name",verbose = True)

In [125]:
llm = OpenAI(temperature=0.7)

prompt_template_items = PromptTemplate(
    input_variables = ['restaurant_name','count'],
    template="Suggest {count} menu items for {restaurant_name}."
)
food_items_chain0 =LLMChain(llm=llm, prompt=prompt_template_items, output_key="menu_items0", verbose = True)

In [132]:
llm = OpenAI(temperature=0.7)

prompt_template_items = PromptTemplate(
    input_variables = ['restaurant_name','count','cuisine'],
    template="Suggest {count} menu items in {cuisine} cusine for restaurant named: {restaurant_name}."
)
food_items_chain1 = LLMChain(llm=llm, prompt=prompt_template_items, output_key="menu_items1", verbose = True)

In [133]:
from langchain.chains import SequentialChain

chain = SequentialChain(
    chains = [name_chain, food_items_chain0, food_items_chain1],
    input_variables = ['cuisine', 'count'],
    output_variables = ['restaurant_name', "menu_items0", "menu_items1"],
    verbose = True
)

In [134]:
print(chain.invoke({"cuisine":"indian", "count":"5"}))



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
I want to open a restaurant for indian food. Suggest a fency name for this.

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Suggest 5 menu items for 

"Spice Palace".

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Suggest 5 menu items in indian cusine for restaurant named: 

"Spice Palace".

> Finished chain.

> Finished chain.
{'cuisine': 'indian', 'count': '5', 'restaurant_name': '\n\n"Spice Palace"', 'menu_items0': '\n\n1. Spicy Chicken Curry: Tender pieces of chicken cooked in a rich, flavorful curry sauce with a kick of spice.\n\n2. Vegetable Samosas: Crispy pastry pockets filled with a medley of seasoned vegetables and served with a spicy dipping sauce.\n\n3. Lamb Vindaloo: Slow-cooked lamb in a fiery vindaloo sauce, perfect for those who love a little heat in their meal.\n\n4. Aloo Gobi: A classic vegetarian dish mad

In [8]:
pip install langchain langchain-community langchain_openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 19.7 MB/s eta 0:00:00


In [10]:
from langchain.chains import LLMChain, SequentialChain
from langchain.prompts import PromptTemplate
from langchain_openai import OpenAI

def generate_restaurant_details(cuisine: str = "Indian", count: int = 5):
    llm = OpenAI(temperature=0.7)

    # Define the prompt template for restaurant name
    prompt_template_name = PromptTemplate(
        input_variables=['cuisine'],
        template="I want to open a restaurant for {cuisine} food. Suggest a fancy name for this."
    )
    name_chain = LLMChain(llm=llm, prompt=prompt_template_name, output_key="restaurant_name")

    # Define the prompt template for menu items without specifying cuisine
    prompt_template_items0 = PromptTemplate(
        input_variables=['restaurant_name', 'count'],
        template="Suggest {count} menu items for {restaurant_name}."
    )
    food_items_chain0 = LLMChain(llm=llm, prompt=prompt_template_items0, output_key="menu_items0")

    # Define the prompt template for menu items with cuisine specified
    prompt_template_items1 = PromptTemplate(
        input_variables=['restaurant_name', 'count', 'cuisine'],
        template="Suggest {count} menu items in {cuisine} cuisine for restaurant named: {restaurant_name}."
    )
    food_items_chain1 = LLMChain(llm=llm, prompt=prompt_template_items1, output_key="menu_items1")

    # Create the sequential chain
    chain = SequentialChain(
        chains=[name_chain, food_items_chain0, food_items_chain1],
        input_variables=['cuisine', 'count'],
        output_variables=['restaurant_name', 'menu_items0', 'menu_items1'],
        verbose=True
    )

    # Execute the chain
    return chain.invoke({"cuisine": cuisine, "count": count})

# Example usage
result = generate_restaurant_details("indian", 5)
print(result)



> Entering new SequentialChain chain...

> Finished chain.
{'cuisine': 'indian', 'count': 5, 'restaurant_name': '\n\n"Spice Palace"', 'menu_items0': '\n\n1. "Spicy Chicken Curry": Tender chunks of chicken cooked in a rich and flavorful curry sauce, served with fragrant basmati rice.\n\n2. "Paneer Tikka Masala": Marinated and grilled paneer (Indian cottage cheese) cooked in a creamy tomato sauce, served with naan bread.\n\n3. "Lamb Vindaloo": A classic Goan-style dish with tender lamb pieces cooked in a spicy and tangy vindaloo sauce, served with steamed rice.\n\n4. "Vegetable Biryani": A colorful and aromatic rice dish made with mixed vegetables, herbs, and spices, served with raita (yogurt sauce).\n\n5. "Tandoori Mixed Grill": A platter of assorted tandoori meats including chicken tikka, lamb seekh kebab, and tandoori shrimp, served with mint chutney and naan bread.', 'menu_items1': '\n\n1. Butter Chicken: Tender chicken pieces simmered in a rich and creamy tomato gravy, flavored wi

# **06. Agents and Tools**


Agents involve an LLM making decisions about which Actions to take, taking that Action, seeing an Observation, and repeating that until done.


When used correctly agents can be extremely powerful. In order to load agents, you should understand the following concepts:

- Tool: A function that performs a specific duty. This can be things like: Google Search, Database lookup, Python REPL, other chains.
- LLM: The language model powering the agent.
- Agent: The agent to use.

Agent is a very powerful concept in LangChain

For example I have to travel from Dubai to Canada, I type this in ChatGPT



---> Give me  two flight options from Dubai to Canada on September 1, 2024 | ChatGPT will not be able to answer because has knowledge till
September 2021



ChatGPT plus has Expedia Plugin, if we enable this plugin it will go to Expedia Plugin and will try to pull information about Flights & it will show the information

SerpApi is a real-time API to access Google search results.

#### Wikipedia and llm-math tool

In [14]:
!pip install wikipedia

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11679 sha256=11f454520a7b99e3d932bf44d7811bfabf460537c0aa1c76f57f5d22b4d31094
  Stored in directory: /root/.cache/pip/wheels/5e/b6/c5/93f3dec388ae76edc830cb42901bb0232504dfc0df02fc50de
Successfully built wikipedia


In [13]:
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain_openai import OpenAI

In [15]:
llm = OpenAI(temperature=0)

In [ ]:
# The tools we'll give the Agent access to. Note that the 'llm-math' tool uses an LLM, so we need to pass that in.
tools = load_tools(["wikipedia", "llm-math"], llm=llm)

# Finally, let's initialize an agent with the tools, the language model, and the type of agent we want to use.
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    # Handle the issue by setting handle_parsing_errors to True.
    # This should allow the agent to handle the parsing errors and continue execution.
    handle_parsing_errors=True
)

# Let's test it out!
agent.run("What was the GDP of US in 2021?")

# **07: Memory**

Chatbot application like ChatGPT, you will notice that it remember past information

In [64]:
from langchain_openai import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

llm = OpenAI(temperature=0.9)

prompt_template_name = PromptTemplate(
    input_variables =['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest a fency name for this."
)

chain = LLMChain(llm=llm,prompt=prompt_template_name)
name = chain.run("Mexican")
print(name)



"La Cantina del Sol"


In [66]:
print(chain.memory)

None


### **ConversationBufferMemory**

We can attach memory to remember all previous conversation

In [62]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()

chain = LLMChain(llm=llm, prompt=prompt_template_name, memory=memory)

In [63]:
name = chain.invoke("Mexican")
print(name)

{'cuisine': 'Mexican', 'history': '', 'text': '\n\n"El Sabor de México" (The Flavor of Mexico)'}


In [55]:
name = chain.invoke("Arabic")
print(name)

{'cuisine': 'Arabic', 'history': 'Human: Arabic\nAI: \n\n"Saffron Oasis"\nHuman: Mexican\nAI: \n"El Sabor Divino" (The Divine Flavor)\nHuman: Arabic\nAI: \n\nAl-Jazeerah Palace ', 'text': '\n"The Arabian Bistro"'}


In [57]:
print(chain.memory)

chat_memory=InMemoryChatMessageHistory(messages=[HumanMessage(content='Mexican', additional_kwargs={}, response_metadata={}), AIMessage(content='\n"El Sabroso Cantina"', additional_kwargs={}, response_metadata={}), HumanMessage(content='Arabic', additional_kwargs={}, response_metadata={}), AIMessage(content='\n\n"Saffron Oasis"', additional_kwargs={}, response_metadata={}), HumanMessage(content='Mexican', additional_kwargs={}, response_metadata={}), AIMessage(content='\n"El Sabor Divino" (The Divine Flavor)', additional_kwargs={}, response_metadata={}), HumanMessage(content='Arabic', additional_kwargs={}, response_metadata={}), AIMessage(content='\n\nAl-Jazeerah Palace ', additional_kwargs={}, response_metadata={}), HumanMessage(content='Arabic', additional_kwargs={}, response_metadata={}), AIMessage(content='\n"The Arabian Bistro"', additional_kwargs={}, response_metadata={})]) k=3


### **ConversationChain**

Conversation buffer memory goes growing endlessly

Just remember last 5 Conversation Chain

Just remember last 10-20 Conversation Chain

In [33]:
from langchain.chains import ConversationChain

convo = ConversationChain(llm=OpenAI(temperature=0.7))
print(convo.prompt.template)

The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI:


<ipython-input-33-ffeb4aaf188e>:3: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :meth:`~RunnableWithMessageHistory: https://python.langchain.com/v0.2/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html` instead.
  convo = ConversationChain(llm=OpenAI(temperature=0.7))


In [34]:
convo.run("Who won the first cricket world cup?")

" The first Cricket World Cup was held in 1975 and was won by the West Indies. The final match was played between the West Indies and Australia at Lord's Cricket Ground in London, England. The West Indies won by 17 runs. The captain of the winning team was Clive Lloyd and the player of the tournament was Australia's Gary Gilmour."

In [35]:
convo.run("How much is 5+5?")

'  5+5 is equal to 10.'

In [36]:
convo.run("Who was the captain of the winning team?")

' The captain of the winning team was Clive Lloyd. He was a left-handed batsman and a medium-pace bowler who played for the West Indies from 1966 to 1985. He led the West Indies to two consecutive World Cup victories in 1975 and 1979. He was also known as the "Supercat" for his athletic fielding abilities.'

In [37]:
print(convo.memory.buffer)

Human: Who won the first cricket world cup?
AI:  The first Cricket World Cup was held in 1975 and was won by the West Indies. The final match was played between the West Indies and Australia at Lord's Cricket Ground in London, England. The West Indies won by 17 runs. The captain of the winning team was Clive Lloyd and the player of the tournament was Australia's Gary Gilmour.
Human: How much is 5+5?
AI:   5+5 is equal to 10.
Human: Who was the captain of the winning team?
AI:  The captain of the winning team was Clive Lloyd. He was a left-handed batsman and a medium-pace bowler who played for the West Indies from 1966 to 1985. He led the West Indies to two consecutive World Cup victories in 1975 and 1979. He was also known as the "Supercat" for his athletic fielding abilities.


### **ConversationBufferWindowMemory**

In [38]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=3)

convo = ConversationChain(
    llm=OpenAI(temperature=0.7),
    memory=memory
)
convo.run("Who won the first cricket world cup?")

<ipython-input-38-199197ae7820>:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferWindowMemory(k=3)


' The first cricket world cup was won by the West Indies in 1975. It was held in England and the West Indies beat Australia by 17 runs in the final match. The captain of the West Indies team was Clive Lloyd and the man of the match was Viv Richards, who scored 138 runs. It was a historic moment for the West Indies team and for cricket as a whole. '

In [39]:
print(convo.memory.buffer)

Human: Who won the first cricket world cup?
AI:  The first cricket world cup was won by the West Indies in 1975. It was held in England and the West Indies beat Australia by 17 runs in the final match. The captain of the West Indies team was Clive Lloyd and the man of the match was Viv Richards, who scored 138 runs. It was a historic moment for the West Indies team and for cricket as a whole. 


In [40]:
convo.run("How much is 5+5?")

' The result of 5+5 is 10.'

In [41]:
print(convo.memory.buffer)

Human: Who won the first cricket world cup?
AI:  The first cricket world cup was won by the West Indies in 1975. It was held in England and the West Indies beat Australia by 17 runs in the final match. The captain of the West Indies team was Clive Lloyd and the man of the match was Viv Richards, who scored 138 runs. It was a historic moment for the West Indies team and for cricket as a whole. 
Human: How much is 5+5?
AI:  The result of 5+5 is 10.


In [42]:
convo.run("Who was the captain of the winning team?")

' The captain of the winning team was Clive Lloyd.'

In [43]:
print(convo.memory.buffer)

Human: Who won the first cricket world cup?
AI:  The first cricket world cup was won by the West Indies in 1975. It was held in England and the West Indies beat Australia by 17 runs in the final match. The captain of the West Indies team was Clive Lloyd and the man of the match was Viv Richards, who scored 138 runs. It was a historic moment for the West Indies team and for cricket as a whole. 
Human: How much is 5+5?
AI:  The result of 5+5 is 10.
Human: Who was the captain of the winning team?
AI:  The captain of the winning team was Clive Lloyd.


In [44]:
convo.run("Who was Clive Lloyd?")

' Clive Lloyd is a former West Indian cricketer who played for the West Indies national team from 1966 to 1985. He is considered one of the greatest cricketers of all time and was known for his strong leadership skills as captain of the West Indies team. He led the team to multiple victories, including the first and second cricket world cups in 1975 and 1979. After his retirement, he went on to serve as a coach and administrator for cricket in the West Indies.'

In [45]:
print(convo.memory.buffer)

Human: How much is 5+5?
AI:  The result of 5+5 is 10.
Human: Who was the captain of the winning team?
AI:  The captain of the winning team was Clive Lloyd.
Human: Who was Clive Lloyd?
AI:  Clive Lloyd is a former West Indian cricketer who played for the West Indies national team from 1966 to 1985. He is considered one of the greatest cricketers of all time and was known for his strong leadership skills as captain of the West Indies team. He led the team to multiple victories, including the first and second cricket world cups in 1975 and 1979. After his retirement, he went on to serve as a coach and administrator for cricket in the West Indies.


#**08: Document Loaders**


In [68]:
!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 5.4 MB/s eta 0:00:00


In [69]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader(
    "/content/Parit_Kansal.pdf",
)
pages = loader.load()

In [77]:
print(pages[0].page_content)

Last Upda ted on 18th No vember 2024 
Parit Kansal
paritk ansal121@gmail.c om | (+91 ) 6 39-895 -0353 
EDUCATION
HARCOURT BUTLER TECHNICAL
UNIVERSITY
B. Tech in Computer Science & 
Engineering 
2025 | K anpur, India 
CGPA:8.132
DEWAN PUBLIC SCHOOL
Senior School 
Ma y 2020 | Hapur, India 
Percentage: 94%
DEWAN PUBLIC SCHOOL
High School 
Ma y 2018 | Hapur, India 
Percentage: 98%
LINKS
Website://paritkansal121.odoo.com
Github://ParitKansal
LinkedIn://paritkansal
LeetCode://ParitKansal
COURSEWORK
UNDERGRADUATE
Compiler Design
Computer Networks
Computer Organization & Architecture
Data Science
Database Management Systems
Numerical & Statistical Techniques
Operating Systems
Web Technology
SKILLS
LANGUAGES
• Python • C++ • C
INTERESTS
• Data Science • ML • Deep Learning
• Computer Vision • NLP
TOOLS AND TECHNOLOGIES
• MS Azure, • MySQL • Power BI • LLM
• Web Scraping • MS Excel-VBA
FRAMEWORKS
• TensorFlow• PyTorch• Sparky
SOFT SKILLS
• Teamwork• Problem Solving
• Communication
EXPERIENCE
EXTI